In [1]:
#nf-ton-oit-v2 e cic ids2017
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### prova chunksize

In [ ]:
nome_dati = '/content/drive/MyDrive/test_dataset_ids/NF-ToN-IoT-v2.csv'
chunksize = 1000
cols=[ 'L4_SRC_PORT',  'L4_DST_PORT',
       'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'IN_PKTS', 'OUT_BYTES', 'OUT_PKTS',
       'TCP_FLAGS', 'CLIENT_TCP_FLAGS', 'SERVER_TCP_FLAGS',
       'FLOW_DURATION_MILLISECONDS', 'DURATION_IN', 'DURATION_OUT', 'MIN_TTL',
       'MAX_TTL', 'LONGEST_FLOW_PKT', 'SHORTEST_FLOW_PKT', 'MIN_IP_PKT_LEN',
       'MAX_IP_PKT_LEN', 'SRC_TO_DST_SECOND_BYTES', 'DST_TO_SRC_SECOND_BYTES',
       'RETRANSMITTED_IN_BYTES', 'RETRANSMITTED_IN_PKTS',
       'RETRANSMITTED_OUT_BYTES', 'RETRANSMITTED_OUT_PKTS',
       'SRC_TO_DST_AVG_THROUGHPUT', 'DST_TO_SRC_AVG_THROUGHPUT',
       'NUM_PKTS_UP_TO_128_BYTES', 'NUM_PKTS_128_TO_256_BYTES',
       'NUM_PKTS_256_TO_512_BYTES', 'NUM_PKTS_512_TO_1024_BYTES',
       'NUM_PKTS_1024_TO_1514_BYTES', 'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT',
       'ICMP_TYPE', 'ICMP_IPV4_TYPE', 'DNS_QUERY_ID', 'DNS_QUERY_TYPE',
       'DNS_TTL_ANSWER', 'FTP_COMMAND_RET_CODE', 'Label', 'Attack']

tabella_dati = pd.read_csv(nome_dati, usecols = cols, chunksize= chunksize ) #TextFileReader --> iterator

for i in tabella_dati:
  print(i.Attack)
  break

0      ransomware
1          Benign
2          Benign
3          Benign
4          Benign
          ...    
995        Benign
996        Benign
997        Benign
998        Benign
999        Benign
Name: Attack, Length: 1000, dtype: object


In [ ]:
# conto il numero di chunk che ho creato
count = 0
for i in tabella_dati:
  count = count + 1
print(count) #->16939 quindi circa 16,9M di righe


In [ ]:
mapping_dict = []

In [ ]:
dizionario = {}
j = 0
for i in tabella_dati:
  for k in i.Attack:
    if (k not in dizionario):
      dizionario[f"{k}"] = j
      j = j + 1
print(dizionario)

{'scanning': 0, 'Benign': 1, 'password': 2, 'ddos': 3, 'xss': 4, 'dos': 5, 'injection': 6, 'mitm': 7, 'ransomware': 8, 'backdoor': 9}


In [ ]:
dizionario = {'scanning': 0, 'Benign': 1, 'password': 2, 'ddos': 3, 'xss': 4, 'dos': 5, 'injection': 6, 'mitm': 7, 'ransomware': 8, 'backdoor': 9}

Scikit Learn prova random forest

Prima di tutto modifico le classi della colonna Attack e le trasformo in numeri


In [ ]:
# If you want to read it again, recreate the TextFileReader or reset it
text_reader = pd.read_csv(nome_dati, chunksize=1000)

for i in text_reader:
  for j in i.Attack:
    j = dizionario[f"{j}"]


In [ ]:
text_reader = pd.read_csv(nome_dati, chunksize=1000)
for i in text_reader:
  print(i)
  break

     IPV4_SRC_ADDR  L4_SRC_PORT    IPV4_DST_ADDR  L4_DST_PORT  PROTOCOL  \
0    192.168.1.193        49235     192.168.1.33         4444         6   
1    192.168.1.193        49228    192.168.1.152         1880         6   
2    192.168.1.152            0    192.168.1.193            0         1   
3    192.168.1.169        65317  239.255.255.250         1900        17   
4     192.168.1.79        60766    192.168.1.255        15600        17   
..             ...          ...              ...          ...       ...   
995   192.168.1.31        41989     192.168.1.49        50000         6   
996   192.168.1.32        60715     192.168.1.49        64680         6   
997   192.168.1.31        40662     192.168.1.49         1078         6   
998   192.168.1.31        51598     192.168.1.49         1107         6   
999   192.168.1.31        42182     192.168.1.49         1022         6   

     L7_PROTO  IN_BYTES  IN_PKTS  OUT_BYTES  OUT_PKTS  ...  TCP_WIN_MAX_IN  \
0         0.0    1553

In [ ]:
#salvare nuova colonna nel file excel


new_column_name = 'Attack_label'

output_file = 'output_file.csv'

# Open the new CSV file for writing
with open(output_file, 'w', newline='') as output_csv:
    # Iterate over chunks
    for i, chunk in enumerate(pd.read_csv(nome_dati, chunksize=1000)):
        # Add a new column to the chunk

        chunk[new_column_name] = chunk.Attack.map(dizionario).fillna(chunk.Attack)

        # Write the modified chunk to the new file
        chunk.to_csv(output_csv, index=False, header=not output_csv.tell())

In [ ]:
#controllo aggiunta colonna
text_reader = pd.read_csv("/content/drive/MyDrive/test_dataset_ids/output_file.csv", chunksize=1000)
for i in text_reader:
  print(i)
  break

     IPV4_SRC_ADDR  L4_SRC_PORT    IPV4_DST_ADDR  L4_DST_PORT  PROTOCOL  \
0    192.168.1.193        49235     192.168.1.33         4444         6   
1    192.168.1.193        49228    192.168.1.152         1880         6   
2    192.168.1.152            0    192.168.1.193            0         1   
3    192.168.1.169        65317  239.255.255.250         1900        17   
4     192.168.1.79        60766    192.168.1.255        15600        17   
..             ...          ...              ...          ...       ...   
995   192.168.1.31        41989     192.168.1.49        50000         6   
996   192.168.1.32        60715     192.168.1.49        64680         6   
997   192.168.1.31        40662     192.168.1.49         1078         6   
998   192.168.1.31        51598     192.168.1.49         1107         6   
999   192.168.1.31        42182     192.168.1.49         1022         6   

     L7_PROTO  IN_BYTES  IN_PKTS  OUT_BYTES  OUT_PKTS  ...  TCP_WIN_MAX_OUT  \
0         0.0    155

In [ ]:
import pandas as pd
import numpy as np
output_file = 'output_file4.csv'
cols=[ 'L4_SRC_PORT',  'L4_DST_PORT',
       'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'IN_PKTS', 'OUT_BYTES', 'OUT_PKTS',
       'TCP_FLAGS', 'CLIENT_TCP_FLAGS', 'SERVER_TCP_FLAGS',
       'FLOW_DURATION_MILLISECONDS', 'DURATION_IN', 'DURATION_OUT', 'MIN_TTL',
       'MAX_TTL', 'LONGEST_FLOW_PKT', 'SHORTEST_FLOW_PKT', 'MIN_IP_PKT_LEN',
       'MAX_IP_PKT_LEN', 'SRC_TO_DST_SECOND_BYTES', 'DST_TO_SRC_SECOND_BYTES',
       'RETRANSMITTED_IN_BYTES', 'RETRANSMITTED_IN_PKTS',
       'RETRANSMITTED_OUT_BYTES', 'RETRANSMITTED_OUT_PKTS',
       'SRC_TO_DST_AVG_THROUGHPUT', 'DST_TO_SRC_AVG_THROUGHPUT',
       'NUM_PKTS_UP_TO_128_BYTES', 'NUM_PKTS_128_TO_256_BYTES',
       'NUM_PKTS_256_TO_512_BYTES', 'NUM_PKTS_512_TO_1024_BYTES',
       'NUM_PKTS_1024_TO_1514_BYTES', 'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT',
       'ICMP_TYPE', 'ICMP_IPV4_TYPE', 'DNS_QUERY_ID', 'DNS_QUERY_TYPE',
       'DNS_TTL_ANSWER', 'FTP_COMMAND_RET_CODE', 'Label']
# Open the new CSV file for writing
with open(output_file, 'w', newline='') as output_csv:
    # Iterate over chunks
    for i, chunk in enumerate(pd.read_csv("/content/drive/MyDrive/test_dataset_ids/output_file.csv", chunksize=1000)):
        # Filter out rows with infinite values in the columns
      for col in chunk.columns:
          chunk = chunk[~chunk[cols].isin([np.inf, -np.inf]).any(axis=1) & (chunk[cols] < 1e15).all(axis=1)]
      # Write the modified chunk to the new file
      chunk.to_csv(output_csv, index=False, header=not output_csv.tell())

In [ ]:
import dask.dataframe as dd
df = dd.read_csv('/content/output_file4.csv')
print(len(df))

16940076


In [ ]:
# salvare il file.csv con colonna aggiunta
from google.colab import files
files.download('output_file4.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# cerco di dividere rispetto indirizzo IP di destinazione
text_reader = pd.read_csv("/content/drive/MyDrive/test_dataset_ids/output_file.csv", chunksize=10000)
d = {}
c = {}
j = 0
for i in text_reader:
  for k in i.IPV4_DST_ADDR:
    if (k not in d):
      d[f"{k}"] = j
      c[j] = f"{k}"
      j = j + 1
print(d)
print(c)

{'192.168.1.33': 0, '192.168.1.152': 1, '192.168.1.193': 2, '239.255.255.250': 3, '192.168.1.255': 4, '172.217.167.67': 5, '192.168.1.37': 6, '192.168.1.190': 7, '196.216.169.10': 8, '193.0.9.9': 9, '91.189.91.139': 10, '224.0.0.251': 11, '52.139.250.253': 12, '8.8.8.8': 13, '192.168.1.1': 14, '216.58.203.106': 15, '13.35.149.60': 16, '192.168.1.180': 17, '192.168.1.49': 18, '192.168.1.194': 19, '192.168.1.46': 20, '192.168.1.186': 21, '192.168.1.195': 22, '192.168.1.169': 23, '192.168.1.32': 24, '192.168.1.133': 25, '101.119.11.11': 26, '224.0.0.252': 27, '224.0.0.1': 28, '101.119.11.17': 29, '91.189.95.3': 30, '84.53.139.64': 31, '205.251.194.102': 32, '192.33.4.12': 33, '192.33.14.30': 34, '149.135.83.134': 35, '184.84.165.131': 36, '220.233.156.30': 37, '169.254.216.168': 38, '103.38.120.36': 39, '224.0.0.2': 40, '103.126.53.123': 41, '150.101.174.187': 42, '203.59.7.250': 43, '144.48.166.166': 44, '203.7.149.149': 45, '192.189.54.33': 46, '216.239.32.10': 47, '192.203.230.10': 48,

In [ ]:
print(j) #indirizzi totali

6868


In [ ]:
# conto quante volte compare ogni indirizzo
text_reader = pd.read_csv("/content/drive/MyDrive/test_dataset_ids/output_file.csv", chunksize=10000)
count = np.zeros(j)
for i in text_reader:
  for k in i.IPV4_DST_ADDR:
    ind = d[k]
    count[ind] += 1
ordine = count.argsort()[::-1][:10]
print(ordine[0:10]) #l'indice dei 10 indirizzi che compaiono più volte nel dataset

[ 22  14   7 872  17   1  18  19  23   2]


In [ ]:
#13,2M sono comunque troppe righe
tot = 0
for i in ordine[0:10]:
  print(c[i], count[i])
  tot += count[i]
print(tot)

192.168.1.195 2118567.0
192.168.1.1 2012390.0
192.168.1.190 1805130.0
192.168.1.184 1653333.0
192.168.1.180 1288737.0
192.168.1.152 1190525.0
192.168.1.49 1016646.0
192.168.1.194 867216.0
192.168.1.169 730266.0
192.168.1.193 538197.0
13221007.0


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Specify the target column and features
target_column = 'Attack_label'
feature_columns = ['L4_SRC_PORT',  'L4_DST_PORT',
       'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'IN_PKTS', 'OUT_BYTES', 'OUT_PKTS',
       'TCP_FLAGS', 'CLIENT_TCP_FLAGS', 'SERVER_TCP_FLAGS',
       'FLOW_DURATION_MILLISECONDS', 'DURATION_IN', 'DURATION_OUT', 'MIN_TTL',
       'MAX_TTL', 'LONGEST_FLOW_PKT', 'SHORTEST_FLOW_PKT', 'MIN_IP_PKT_LEN',
       'MAX_IP_PKT_LEN', 'SRC_TO_DST_SECOND_BYTES', 'DST_TO_SRC_SECOND_BYTES',
       'RETRANSMITTED_IN_BYTES', 'RETRANSMITTED_IN_PKTS',
       'RETRANSMITTED_OUT_BYTES', 'RETRANSMITTED_OUT_PKTS',
       'SRC_TO_DST_AVG_THROUGHPUT', 'DST_TO_SRC_AVG_THROUGHPUT',
       'NUM_PKTS_UP_TO_128_BYTES', 'NUM_PKTS_128_TO_256_BYTES',
       'NUM_PKTS_256_TO_512_BYTES', 'NUM_PKTS_512_TO_1024_BYTES',
       'NUM_PKTS_1024_TO_1514_BYTES', 'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT',
       'ICMP_TYPE', 'ICMP_IPV4_TYPE', 'DNS_QUERY_ID', 'DNS_QUERY_TYPE',
       'DNS_TTL_ANSWER', 'FTP_COMMAND_RET_CODE', 'Label']

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
arr = np.array([5, 68, 89, 98]) # -> chunk che presentano un valore che da errore
chunksize = 100000
for i, chunk in tqdm(enumerate(pd.read_csv("/content/drive/MyDrive/test_dataset_ids/output_file.csv", chunksize=chunksize))):
    if( i not in arr):
      X = chunk[feature_columns]
      y = chunk[target_column]


      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


      rf_classifier.fit(X_train, y_train) # così crea una nuova foresta ad ogni chiamata di fit non va bene

      if( i % 99 == 0):
        # Predict on the test set and evaluate accuracy
        y_pred = rf_classifier.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f'Chunk {i + 1} - Accuracy: {accuracy:.4f}')

In [ ]:
total_chunks = 170 #se ho chunk di dimensione 100k

shuffled_indices = np.random.permutation(total_chunks)# problema righe con valori non validi

In [ ]:
# cerco chunk con tot numero righe che corrispondono ad una determinata classe
chunksize = 100000
ind = 1
for i, chunk in tqdm(enumerate(pd.read_csv("/content/drive/MyDrive/test_dataset_ids/output_file.csv", chunksize=chunksize))):
  tot = 0
  for j in chunk.Attack_label:
    if(j == ind):
      tot += 1
  if (tot > 40000): #se ho più di n righe di quella classe, prendi il chunk
    print (i, ind, tot)
    #ind += 1

In [ ]:
# controllo distibuzione dei dati
arr = np.zeros(10)
for i in pd.read_csv("/content/drive/MyDrive/test_dataset_ids/output_file.csv", chunksize=chunksize):
  for j in i.Attack_label:
    arr[j] += 1
print(arr)
arr1 = np.zeros(10)
for i in range(arr.size):
  arr1[i] = ((arr[i]/16939000)*100)
#stampo percentuale rispetto al totale delle righe
for i in range(arr1.size):
  print(f"{arr1[i]:.2f}")

[3.781419e+06 6.099469e+06 1.153323e+06 2.026234e+06 2.455020e+06
 7.126090e+05 6.844650e+05 7.723000e+03 3.425000e+03 1.680900e+04]
22.32
36.01
6.81
11.96
14.49
4.21
4.04
0.05
0.02
0.10


In [ ]:
#distribuzione dati del subset creato
subset = [1,6,156,108,124,144,169,162]
chunksize = 100000
arr = np.zeros(10)
for i in subset:
  start_index = chunksize * i
  end_index = start_index + chunksize
  chunk = pd.read_csv("/content/drive/MyDrive/test_dataset_ids/output_file.csv", skiprows=range(1, start_index), nrows=chunksize)
  for j in chunk.Attack_label:
    arr[j] += 1

print(arr)
arr1 = np.zeros(10)
for i in range(arr.size):
  arr1[i] = ((arr[i]/800000)*100)
#stampo percentuale rispetto al totale delle righe
for i in range(arr1.size):
  print(f"{arr1[i]:.2f}")

[ 91804. 233104.  88376. 147327.  91849.  30766.  48030.   2126.   3392.
   3723.]
11.48
29.14
11.05
18.42
11.48
3.85
6.00
0.27
0.42
0.47


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, warm_start= True) #così ad ogni train aggiungo al modello precedente
subset = [1,6,156,108,124,144,169,162]
k = 2
for i in subset:
    start_index = chunksize * i
    end_index = start_index + chunksize
    chunk = pd.read_csv("/content/drive/MyDrive/test_dataset_ids/output_file.csv", skiprows=range(1, start_index), nrows=chunksize)
    X = chunk[feature_columns]
    y = chunk[target_column]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    rf_classifier.classes_ = [i for i in range(10)]

    rf_classifier.fit(X_train, y_train)
    _ = rf_classifier.set_params(n_estimators=100*k, warm_start=True) #così aumento il numero di alberi di 100
    k += 1
    # # Prediction
    # y_pred = rf_classifier.predict(X_test)
    # accuracy = accuracy_score(y_test, y_pred)

    #print(f'Iter {k-2} - Accuracy: {accuracy:.4f}')

In [ ]:
total_chunks = 170 #se ho chunk di dimensione 100k

shuffled_indices = np.random.permutation(total_chunks)# potrebbe avere un problema di righe con valori non validi

for i in shuffled_indices[0:5]:
  start_index = chunksize * i
  end_index = start_index + chunksize
  chunk = pd.read_csv("/content/drive/MyDrive/test_dataset_ids/output_file.csv", skiprows=range(1, start_index), nrows=chunksize)
  X = chunk[feature_columns]
  y = chunk[target_column]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  y_pred = rf_classifier.predict(X_test)
  #accuracy = accuracy_score(y, y_pred)
  #print(f'Iter {i} - Accuracy: {accuracy:.4f}')

ValueError: operands could not be broadcast together with shapes (20000,8) (20000,7) (20000,8) 

In [ ]:
chunks = []
subset = [1,6,156,108,124,144,169,162]
chunksize = 100000

for i in subset:
    start_index = chunksize * i
    end_index = start_index + chunksize
    chunk = pd.read_csv("/content/drive/MyDrive/test_dataset_ids/output_file.csv", skiprows=range(1, start_index), nrows=chunksize)

    chunks.append(chunk)

concatenated_df = pd.concat(chunks, ignore_index=True)

In [ ]:
concatenated_df.shape

(740497, 46)

In [ ]:
feature_columns = ['L4_SRC_PORT',  'L4_DST_PORT',
       'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'IN_PKTS', 'OUT_BYTES', 'OUT_PKTS',
       'TCP_FLAGS', 'CLIENT_TCP_FLAGS', 'SERVER_TCP_FLAGS',
       'FLOW_DURATION_MILLISECONDS', 'DURATION_IN', 'DURATION_OUT', 'MIN_TTL',
       'MAX_TTL', 'LONGEST_FLOW_PKT', 'SHORTEST_FLOW_PKT', 'MIN_IP_PKT_LEN',
       'MAX_IP_PKT_LEN', 'SRC_TO_DST_SECOND_BYTES', 'DST_TO_SRC_SECOND_BYTES',
       'RETRANSMITTED_IN_BYTES', 'RETRANSMITTED_IN_PKTS',
       'RETRANSMITTED_OUT_BYTES', 'RETRANSMITTED_OUT_PKTS',
       'SRC_TO_DST_AVG_THROUGHPUT', 'DST_TO_SRC_AVG_THROUGHPUT',
       'NUM_PKTS_UP_TO_128_BYTES', 'NUM_PKTS_128_TO_256_BYTES',
       'NUM_PKTS_256_TO_512_BYTES', 'NUM_PKTS_512_TO_1024_BYTES',
       'NUM_PKTS_1024_TO_1514_BYTES', 'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT',
       'ICMP_TYPE', 'ICMP_IPV4_TYPE', 'DNS_QUERY_ID', 'DNS_QUERY_TYPE',
       'DNS_TTL_ANSWER', 'FTP_COMMAND_RET_CODE', 'Label']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
X = concatenated_df[feature_columns]
y = concatenated_df["Attack_label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

 Accuracy: 0.9907


### prova con dask

In [2]:
#prova con dask:
import dask.dataframe as dd
df = dd.read_csv('/content/drive/MyDrive/test_dataset_ids/output_file.csv')
df.head(10)

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label,Attack,Attack_label
0,192.168.1.193,49235,192.168.1.33,4444,6,0.000,155392,202,34552,149,...,4805,0,0,0,0,0,0,1,ransomware,8
1,192.168.1.193,49228,192.168.1.152,1880,6,0.000,1600,40,35741,65,...,237,0,0,0,0,0,0,0,Benign,1
2,192.168.1.152,0,192.168.1.193,0,1,0.000,212,2,0,0,...,0,771,3,0,0,0,0,0,Benign,1
3,192.168.1.169,65317,239.255.255.250,1900,17,0.000,165,1,0,0,...,0,0,0,0,0,0,0,0,Benign,1
4,192.168.1.79,60766,192.168.1.255,15600,17,0.000,63,1,0,0,...,0,0,0,0,0,0,0,0,Benign,1
5,192.168.1.193,137,192.168.1.33,137,17,0.000,234,3,0,0,...,0,0,0,0,0,0,0,1,ransomware,8
6,192.168.1.33,0,192.168.1.193,0,1,0.000,318,3,0,0,...,0,771,3,0,0,0,0,0,Benign,1
7,192.168.1.133,52729,239.255.255.250,1900,17,0.000,459,3,0,0,...,0,0,0,0,0,0,0,0,Benign,1
8,192.168.1.193,49233,172.217.167.67,80,6,7.126,121,3,92,2,...,242,0,0,0,0,0,0,0,Benign,1
9,192.168.1.79,44596,239.255.255.250,15600,17,0.000,63,1,0,0,...,0,0,0,0,0,0,0,0,Benign,1


In [3]:
X = df.drop(columns=['IPV4_SRC_ADDR', 'IPV4_DST_ADDR', 'SRC_TO_DST_SECOND_BYTES', 'DST_TO_SRC_SECOND_BYTES' , 'Attack','Attack_label'])
#X = df.drop(columns=['IPV4_SRC_ADDR', 'IPV4_DST_ADDR','Attack','Attack_label'])
Y = df['Attack_label']

In [ ]:
X.head(5)

,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
0,49235,4444,6,0.0,155392,202,34552,149,24,24,...,95,45555,4805,0,0,0,0,0,0,1
1,49228,1880,6,0.0,1600,40,35741,65,24,16,...,6,16425,237,0,0,0,0,0,0,0
2,0,0,1,0.0,212,2,0,0,0,0,...,0,0,0,771,3,0,0,0,0,0
3,65317,1900,17,0.0,165,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,60766,15600,17,0.0,63,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
pip install dask-ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.8/149.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00


In [5]:
from dask_ml.model_selection import train_test_split

# train_x, train_y, test_x, test_y = train_test_split(X, Y, test_size=0.2, shuffle = True, random_state=42)
# train_x.head(10)

X_drop, X_keep, y_drop, y_keep = train_test_split(X, Y, test_size=0.2, shuffle = True, random_state=42)

In [6]:
train_x, test_x, train_y, test_y = train_test_split(X_keep, y_keep, test_size=0.2, shuffle = True, random_state=42)

In [7]:
import xgboost as xgb

def transform_dataset_to_dmatrix(data_x, data_y) :
    """Transform dataset to DMatrix format for xgboost."""
    new_data = xgb.DMatrix(data_x, label=data_y)
    pandas_df = data_y.compute()
    return new_data, pandas_df

In [8]:
# Reformat data to DMatrix for xgboost
train_dmatrix, train_y= transform_dataset_to_dmatrix(train_x, train_y)
valid_dmatrix, valid_y= transform_dataset_to_dmatrix(test_x, test_y)


In [13]:

params = {

    "objective": "multi:softprob",
    "eta": 0.1,  # lr
    "max_depth": 8,
    "colsample_bynode": 0.8,
    "eval_metric": "mlogloss",
    "nthread": 16,
    "num_class" : 10,
    "num_parallel_tree": 100, #num_parallel_tree should be set to the size of the forest being trained.
    "subsample": 1,
    "tree_method": "hist",
}

bst = xgb.train(params, train_dmatrix, num_boost_round=5) #should be set to 1 to prevent XGBoost from boosting multiple random forests


In [14]:
from sklearn.metrics import accuracy_score
#precision
#from sklearn.metrics import f1_score

y_pred = bst.predict(valid_dmatrix)
r = []
for i in y_pred:
    r.append(i.argmax())
accuracy = accuracy_score(valid_y, r)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.9724


In [ ]:
#from sklearn.model_selection import RandomizedSearchCV
#learning_rate = [0.001, 0.01, 0.05, 0.1]
learning_rate = [0.1, 0.2, 0.5]
num_parallel_tree = [1,10,30]
max_depth = [3, 5, 8]
num_boost_round = [1, 5, 10]

for i in learning_rate:
  for j in num_parallel_tree:
    for k in max_depth:
      for h in num_boost_round:
        print(f"lr:{i}, npt:{j}, md:{k}, nbr:{h}")
        params = {
          "objective": "multi:softprob",
          "eta": i,  # lr
          "max_depth": k,
          "eval_metric": "mlogloss",
          "nthread": 16,
          "num_class" : 10,
          "num_parallel_tree": j, #num_parallel_tree should be set to the size of the forest being trained.
          "subsample": 1,
          "tree_method": "hist",
        }

        bst = xgb.train(params, train_dmatrix, num_boost_round=h)
        y_pred = bst.predict(valid_dmatrix)
        r = []
        for l in y_pred:
            r.append(l.argmax())
        accuracy = accuracy_score(valid_y, r)
        print(f'Accuracy: {accuracy:.4f}')



lr:0.1, npt:1, md:3, nbr:1
Accuracy: 0.9314
lr:0.1, npt:1, md:3, nbr:5
Accuracy: 0.9452
lr:0.1, npt:1, md:3, nbr:10
Accuracy: 0.9475
lr:0.1, npt:1, md:5, nbr:1
Accuracy: 0.9557
lr:0.1, npt:1, md:5, nbr:5
Accuracy: 0.9600
lr:0.1, npt:1, md:5, nbr:10
Accuracy: 0.9617
lr:0.1, npt:1, md:8, nbr:1
Accuracy: 0.9673
lr:0.1, npt:1, md:8, nbr:5
Accuracy: 0.9694
lr:0.1, npt:1, md:8, nbr:10
Accuracy: 0.9728
lr:0.1, npt:10, md:3, nbr:1
Accuracy: 0.9314
lr:0.1, npt:10, md:3, nbr:5
Accuracy: 0.9452
lr:0.1, npt:10, md:3, nbr:10
Accuracy: 0.9475
lr:0.1, npt:10, md:5, nbr:1
Accuracy: 0.9557
lr:0.1, npt:10, md:5, nbr:5
Accuracy: 0.9600
lr:0.1, npt:10, md:5, nbr:10
Accuracy: 0.9617
lr:0.1, npt:10, md:8, nbr:1
Accuracy: 0.9673
lr:0.1, npt:10, md:8, nbr:5
Accuracy: 0.9694
lr:0.1, npt:10, md:8, nbr:10
Accuracy: 0.9728
lr:0.1, npt:30, md:3, nbr:1
Accuracy: 0.9314
lr:0.1, npt:30, md:3, nbr:5
Accuracy: 0.9452
lr:0.1, npt:30, md:3, nbr:10
Accuracy: 0.9475
lr:0.1, npt:30, md:5, nbr:1
Accuracy: 0.9557
lr:0.1, npt:

In [ ]:
# prova load model sbilanciato
loaded_model = xgb.Booster()
loaded_model.load_model('/content/drive/MyDrive/test_dataset_ids/modelli/global_model.json')

In [ ]:
y_pred = loaded_model.predict(valid_dmatrix)
r = []
for l in y_pred:
    r.append(l.argmax())
accuracy = accuracy_score(valid_y, r)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.6551


In [ ]:
# prova load model bilanciato
loaded_model_bilanciato = xgb.Booster()
loaded_model_bilanciato.load_model('/content/drive/MyDrive/test_dataset_ids/modelli/global_model_bilanciato.json')

In [ ]:
y_pred = loaded_model_bilanciato.predict(valid_dmatrix)
r = []
for l in y_pred:
    r.append(l.argmax())
accuracy = accuracy_score(valid_y, r)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.9681


### Divisione dataset secondo colonna 'IPV4_DST_ADDR'

In [ ]:
import dask.dataframe as dd

In [ ]:
df = dd.read_csv('/content/drive/MyDrive/test_dataset_ids/output_file.csv')

In [ ]:
value_counts_attack = df['IPV4_DST_ADDR'].value_counts().compute()
print(value_counts_attack)

192.168.1.195     2118567
192.168.1.1       2012390
192.168.1.190     1805130
192.168.1.184     1653333
192.168.1.180     1288737
                   ...   
221.176.131.54          1
139.177.139.85          1
190.151.44.192          1
190.14.135.230          1
144.35.68.125           1
Name: IPV4_DST_ADDR, Length: 6868, dtype: int64


In [ ]:

partitions = df.set_index('IPV4_DST_ADDR')

partition_sizes = partitions.map_partitions(lambda df: df.shape[0])

print(partition_sizes)

Dask Series Structure:
npartitions=33
0.0.0.0          int64
101.119.11.10      ...
                 ...  
96.7.49.67         ...
99.99.99.142       ...
dtype: int64
Dask Name: lambda, 9 graph layers


In [ ]:
largest_partition_sizes = partition_sizes.compute().nlargest(20)
print(largest_partition_sizes)

15    2149733
5     2012400
12    1805160
10    1653333
22    1328198
9     1288737
7     1190525
14     867216
8      730266
13     538197
11     457054
21     454179
18     352189
3      342414
17     243737
30     225428
4      209544
28     156593
27     142681
23     121754
dtype: int64


In [ ]:
p = largest_partition_sizes.index

In [ ]:
lista = partitions.divisions
for i in p:
  print(lista[i])

192.168.1.195
192.168.1.1
192.168.1.190
192.168.1.184
192.168.1.49
192.168.1.180
192.168.1.152
192.168.1.194
192.168.1.169
192.168.1.193
192.168.1.186
192.168.1.46
192.168.1.31
176.28.50.165
192.168.1.30
35.156.66.156
18.184.104.180
192.168.35.57
192.168.35.229
192.168.35.100


In [ ]:
value_counts_attack[0:20]

192.168.1.195     2118567
192.168.1.1       2012390
192.168.1.190     1805130
192.168.1.184     1653333
192.168.1.180     1288737
192.168.1.152     1190525
192.168.1.49      1016646
192.168.1.194      867216
192.168.1.169      730266
192.168.1.193      538197
192.168.1.186      457054
192.168.1.46       454179
176.28.50.165      342235
192.168.1.79       310575
192.168.1.30       243737
192.168.1.31       238884
52.28.231.150      149165
18.194.169.124     146956
192.168.1.38        77513
192.168.1.32        71900
Name: IPV4_DST_ADDR, dtype: int64

In [ ]:
partitions.head(5)

,IPV4_SRC_ADDR,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,...,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label,Attack,Attack_label
IPV4_DST_ADDR,,,,,,,,,,,,,,,,,,,,,
0.0.0.0,0.0.0.0,64420,53,17,5.212,176,2,0,0,0,...,0,0,0,50908,28,0,0,0,Benign,1
0.0.0.0,0.0.0.0,12337,53,17,0.000,87,1,0,0,0,...,0,0,0,43398,43,0,0,0,Benign,1
0.0.0.0,0.0.0.0,1324,53,17,0.000,76,1,0,0,0,...,0,0,0,46071,2,0,0,0,Benign,1
0.0.0.0,0.0.0.0,25307,53,17,0.000,80,1,0,0,0,...,0,0,0,62012,48,0,0,0,Benign,1
0.0.0.0,0.0.0.0,24372,53,17,0.000,80,1,0,0,0,...,0,0,0,56359,48,0,0,0,Benign,1


In [ ]:
# from tqdm import tqdm
# diz = []
# cont = 0
# for i in tqdm(lista):
#   p1 = partitions.loc[i].compute()
#   diz.append((i, cont, p1.shape[0]))
#   cont += 1

In [ ]:
# print(diz)

In [ ]:
partitions.divisions[13]

'192.168.1.194'

In [ ]:
p1 = partitions.loc['192.168.1.194']

In [ ]:
p1.shape

(Delayed('int-88d1118e-c959-477c-9b35-d5a8b196dba3'), 45)